In [1]:
!pip install flask flask_sqlalchemy werkzeug validators


In [2]:
home_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Home</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="bg-light">

<div class="container text-center mt-5">
    <h1 class="mb-4">🔗 URL Shortener</h1>
    <p class="lead">Shorten your links easily & save history.</p>

    <a href="/login" class="btn btn-primary m-2">Login</a>
    <a href="/signup" class="btn btn-success m-2">Signup</a>
</div>

</body>
</html>
"""
with open("templates/home.html", "w", encoding="utf-8") as f:
    f.write(home_html)

print("✅ home.html created")


✅ home.html created


In [3]:
signup_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Signup</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="bg-light">

<div class="container mt-5 col-md-4">
    <h2 class="text-center">Signup</h2>

    {% with messages = get_flashed_messages(with_categories=true) %}
    {% for category, msg in messages %}
        <div class="alert alert-{{category}}">{{msg}}</div>
    {% endfor %}
    {% endwith %}

    <form method="POST">
        <input type="text" name="username" class="form-control my-2" placeholder="Username" required>
        <input type="password" name="password" class="form-control my-2" placeholder="Password" required>
        <button class="btn btn-success w-100">Signup</button>
    </form>

    <p class="text-center mt-3">
        Already have an account? <a href="/login">Login</a>
    </p>
</div>

</body>
</html>
"""
with open("templates/signup.html", "w", encoding="utf-8") as f:
    f.write(signup_html)

print("✅ signup.html created")


✅ signup.html created


In [4]:
login_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="bg-light">

<div class="container mt-5 col-md-4">
    <h2 class="text-center">Login</h2>

    {% with messages = get_flashed_messages(with_categories=true) %}
    {% for category, msg in messages %}
        <div class="alert alert-{{category}} mt-2">{{msg}}</div>
    {% endfor %}
    {% endwith %}

    <form method="POST">
        <input type="text" name="username" class="form-control my-2" placeholder="Username" required>
        <input type="password" name="password" class="form-control my-2" placeholder="Password" required>
        <button class="btn btn-primary w-100">Login</button>
    </form>

    <p class="text-center mt-3">
        No account? <a href="/signup">Signup</a>
    </p>
</div>

</body>
</html>
"""
with open("templates/login.html", "w", encoding="utf-8") as f:
    f.write(login_html)

print("✅ login.html created")


✅ login.html created


In [5]:
dashboard_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Dashboard</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="bg-light">

<div class="container mt-5 col-md-6">
    <h2 class="text-center mb-4">Welcome {{session.username}} 👋</h2>

    <div class="d-flex justify-content-between">
        <a href="/history" class="btn btn-warning">History</a>
        <a href="/logout" class="btn btn-danger">Logout</a>
    </div>

    {% with messages = get_flashed_messages(with_categories=true) %}
    {% for category, msg in messages %}
        <div class="alert alert-{{category}} mt-3">{{msg}}</div>
    {% endfor %}
    {% endwith %}

    <form method="POST" class="mt-4">
        <input type="text" name="original_url" class="form-control" placeholder="Enter URL" required>
        <button class="btn btn-success w-100 mt-3">Shorten URL</button>
    </form>

    {% if short_url %}
    <div class="mt-4">
        <label>Shortened URL</label>
        <div class="input-group">
            <input type="text" id="shortUrl" class="form-control" value="{{short_url}}" readonly>
            <button class="btn btn-primary" onclick="copyText()">Copy</button>
        </div>
    </div>
    {% endif %}
</div>

<script>
function copyText() {
    let copyField = document.getElementById("shortUrl");
    copyField.select();
    copyField.setSelectionRange(0, 99999);
    navigator.clipboard.writeText(copyField.value);
    alert("Copied: " + copyField.value);
}
</script>

</body>
</html>
"""
with open("templates/dashboard.html", "w", encoding="utf-8") as f:
    f.write(dashboard_html)

print("✅ dashboard.html created")




✅ dashboard.html created


In [6]:
history_html = """
<!DOCTYPE html>
<html>
<head>
    <title>History</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="bg-light">

<div class="container mt-5">
    <h2 class="text-center mb-4">URL History</h2>

    <a href="/dashboard" class="btn btn-secondary mb-3">⬅ Back</a>

    <table class="table table-bordered table-striped">
        <thead>
            <tr>
                <th>Original URL</th>
                <th>Shortened URL</th>
                <th>Created</th>
            </tr>
        </thead>
        <tbody>
        {% for u in urls %}
            <tr>
                <td style="word-break: break-all">{{u.original_url}}</td>
                <td>
                    <a href="/{{u.short_code}}" target="_blank">
                        {{request.host_url}}{{u.short_code}}
                    </a>
                </td>
                <td>{{u.created_at.strftime("%d-%m-%Y %H:%M")}}</td>
            </tr>
        {% endfor %}
        </tbody>
    </table>
</div>

</body>
</html>
"""
with open("templates/history.html", "w", encoding="utf-8") as f:
    f.write(history_html)




print("✅ history.html created")


✅ history.html created


In [8]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime
import string, random
import validators

app = Flask(__name__)
app.config["SECRET_KEY"] = "supersecretkey123"
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)

# ---- MODELS ----
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(200), nullable=False)

class URL(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    original_url = db.Column(db.Text, nullable=False)
    short_code = db.Column(db.String(10), unique=True, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

    user_id = db.Column(db.Integer, db.ForeignKey("user.id"), nullable=False)
    user = db.relationship("User", backref="urls")

with app.app_context():
    db.create_all()

# ---- HELPERS ----
def generate_short_code(length=6):
    chars = string.ascii_letters + string.digits
    while True:
        code = "".join(random.choice(chars) for _ in range(length))
        if not URL.query.filter_by(short_code=code).first():
            return code

def login_required():
    return "user_id" in session

# --- ROUTES ---
@app.route("/")
def home():
    return render_template("home.html")

@app.route("/signup", methods=["GET","POST"])
def signup():
    if request.method == "POST":
        username = request.form.get("username")
        password = request.form.get("password")

        if User.query.filter_by(username=username).first():
            flash("Username already exists!", "danger")
            return redirect(url_for("signup"))

        hashed_pw = generate_password_hash(password)
        user = User(username=username, password=hashed_pw)

        db.session.add(user)
        db.session.commit()

        flash("Signup successful! Please login.", "success")
        return redirect(url_for("login"))

    return render_template("signup.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        username = request.form.get("username")
        password = request.form.get("password")

        user = User.query.filter_by(username=username).first()
        if not user or not check_password_hash(user.password, password):
            flash("Invalid credentials!", "danger")
            return redirect(url_for("login"))

        session["user_id"] = user.id
        session["username"] = user.username
        flash("Login successful!", "success")
        return redirect(url_for("dashboard"))

    return render_template("login.html")

@app.route("/logout")
def logout():
    session.clear()
    flash("Logged out!", "success")
    return redirect(url_for("home"))

@app.route("/dashboard", methods=["GET","POST"])
def dashboard():
    if not login_required():
        flash("Please login first!", "danger")
        return redirect(url_for("login"))

    short_url = None

    if request.method == "POST":
        original_url = request.form.get("original_url")

        #  URL Validation
        if not validators.url(original_url):
            flash("Invalid URL! Please enter a correct URL.", "danger")
            return redirect(url_for("dashboard"))

        code = generate_short_code()

        url_entry = URL(
            original_url=original_url,
            short_code=code,
            user_id=session["user_id"]
        )
        db.session.add(url_entry)
        db.session.commit()

        short_url = request.host_url + code

    return render_template("dashboard.html", short_url=short_url)

@app.route("/history")
def history():
    if not login_required():
        flash("Please login first!", "danger")
        return redirect(url_for("login"))

    urls = URL.query.filter_by(user_id=session["user_id"]).order_by(URL.created_at.desc()).all()
    return render_template("history.html", urls=urls)

@app.route("/<short_code>")
def redirect_url(short_code):
    url_obj = URL.query.filter_by(short_code=short_code).first()
    if not url_obj:
        return "<h2>Short URL not found!</h2>", 404
    return redirect(url_obj.original_url)


In [ ]:
app.run(debug=True, port=5000, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Jan/2026 16:32:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2026 16:32:30] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2026 16:32:36] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [18/Jan/2026 16:32:36] "GET /dashboard HTTP/1.1" 200 -
